In [ ]:
input_str = "Step on no petS!2"

temp = input_str.replace(input_str, "2")
print(temp)

special_chars = "!@#$%%^&*()_+{}[]:;\"'<>,.>/?|-=" #예시
result = input_str.replace(" ", "").lower()
for char in input_str:
  if char in special_chars:
    result = result.replace(char,'')

print(result)
#result = result.replace(" ", "").lower()
answer = ""

for i in range(len(input_str)-1,-1,-1):
  if input_str[i] in special_chars:
    continue
  answer +=input_str[i]

print(answer)
print(result)

reversed_result = answer.replace(" ","").lower()
for char in answer:
  if char in special_chars:
    reversed_result = reversed_result.replace(char, '')

print(reversed_result)
print(result == reversed_result)

2
steponnopets2
2Step on no petS
steponnopets2
2steponnopets
False


In [ ]:
# 4.	Data preparation
#A.	You can download the dataset directly in you Python code like above.
#B.	Use print(df.head())function to see the first few lines of the data.

import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv", filename="spam.csv")
df = pd.read_csv('spam.csv', encoding='latin1', usecols=[0, 1])

df.columns = ['Label', 'Message']

print(df.head())


  Label                                            Message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [ ]:

import re

# Function to split messages into words
def tokenize(message):
  message = message.lower()# Convert to lowercase 대문자를 소문자로 변경

  words = re.findall(r'\b\w+\b', message)# Extract words

  return words

In [ ]:

# Separate spam and non-spam messages
spam_messages = df[df['Label'] == 'spam']['Message']
non_spam_messages = df[df['Label'] == 'ham']['Message']

In [ ]:
# Count the number of spam messages
spam_count = spam_messages.shape[0]
non_spam_count = non_spam_messages.shape[0]
# Print the result
print(f"Total number of spam messages: {spam_count}")
print(f"Total number of non_spam messages: {non_spam_count}")


Total number of spam messages: 747
Total number of non_spam messages: 4825


8.	Question 1: How to get the number of non-spam(ham) messages?

->->
non_spam_count = non_spam_messages.shape[0]
print(f"Total number of non_spam messages: {non_spam_count}")

In [ ]:
from collections import Counter

# Calculate word frequency
spam_words = Counter()
non_spam_words = Counter()

for message in spam_messages:
    spam_words.update(tokenize(message))

for message in non_spam_messages:
    non_spam_words.update(tokenize(message))


In [ ]:
# Print the most common words
print("Most common words in spam messages:")
print(spam_words.most_common(30))

print("\nMost common words in non-spam messages:")
print(non_spam_words.most_common(30))


Most common words in spam messages:
[('to', 688), ('a', 377), ('call', 355), ('å', 299), ('you', 297), ('your', 264), ('free', 224), ('2', 206), ('the', 206), ('for', 203), ('now', 199), ('or', 188), ('u', 174), ('txt', 163), ('is', 158), ('on', 144), ('ur', 144), ('4', 137), ('have', 135), ('from', 131), ('mobile', 127), ('text', 125), ('and', 122), ('stop', 121), ('claim', 113), ('1', 111), ('with', 109), ('reply', 104), ('www', 98), ('of', 95)]

Most common words in non-spam messages:
[('i', 2940), ('you', 1943), ('to', 1554), ('the', 1122), ('a', 1056), ('u', 1018), ('and', 857), ('in', 818), ('me', 772), ('my', 750), ('is', 732), ('it', 711), ('that', 551), ('of', 525), ('for', 501), ('s', 478), ('have', 440), ('can', 439), ('so', 435), ('but', 434), ('your', 417), ('not', 415), ('are', 414), ('m', 405), ('on', 393), ('do', 384), ('at', 378), ('t', 375), ('if', 354), ('we', 354)]


11.	Question 2: Do you think is it meaningful to use the most common words in each message category to make a spam filter? Explain why.

->->
No, it's unmeaningful. Because 스팸 메시지에서의 가장 흔한 30개 단어와, 비스팸 메시지에서의 가장 흔한 30개 단어를 동일비교 해봤을 때, 12개의 단어가 나왔다. 그렇다면 그 12개의 단어로 이루어진 문장이라면, 그것은 스팸메시지인지 비스팸메시지인지 구분할 수 없다. 그러므로 이 방식은 spam filter를 만들려는 방식으로는 한계가 있다.

In [ ]:
# Identify words that are more frequent in spam than in non-spam
# Compare the frequency of each word in spam and non-spam
unique_words = set(list(spam_words.keys()) + list(non_spam_words.keys()))

spam_dominant_words = {}

for word in unique_words:
    spam_freq = spam_words[word] #value값이 나온다. 즉 숫자.
    non_spam_freq = non_spam_words[word] #value값이 나온다. 즉 숫자.

    p_w = spam_freq / spam_count
    q_w = non_spam_freq / non_spam_count
    #spam_greq > non_spam_freq *2
    #if spam_freq > non_spam_freq * 20 and q_w !=0 :
    if p_w > q_w *2 and q_w!=0 :
        spam_dominant_words[word]  = spam_freq#p_w / (p_w+q_w) #spam_freq
        #print(f"{word} : {p_w / (p_w+q_w):.3f}") #f"{number:.3f}"
       #spam_dominant_words[word] = p_w / (p_w + q_w)

print(sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True))



for word, freq in sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True):
   print(f"{word}: {freq}")
print(len(sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True)))


[('to', 688), ('a', 377), ('call', 355), ('å', 299), ('your', 264), ('free', 224), ('2', 206), ('for', 203), ('now', 199), ('or', 188), ('txt', 163), ('ur', 144), ('on', 144), ('4', 137), ('from', 131), ('mobile', 127), ('text', 125), ('stop', 121), ('1', 111), ('with', 109), ('reply', 104), ('www', 98), ('this', 89), ('our', 85), ('only', 84), ('cash', 76), ('won', 76), ('uk', 74), ('send', 70), ('new', 69), ('nokia', 67), ('win', 64), ('urgent', 63), ('week', 60), ('50', 57), ('service', 56), ('c', 56), ('contact', 56), ('please', 55), ('co', 54), ('com', 54), ('msg', 54), ('16', 51), ('who', 49), ('customer', 49), ('per', 46), ('phone', 45), ('3', 45), ('chat', 44), ('been', 43), ('min', 43), ('100', 40), ('draw', 39), ('mins', 36), ('latest', 36), ('sms', 36), ('line', 35), ('camera', 35), ('message', 34), ('every', 33), ('receive', 33), ('wk', 32), ('landline', 32), ('holiday', 32), ('video', 32), ('shows', 31), ('find', 31), ('rate', 30), ('apply', 30), ('box', 30), ('number', 30

13.	Question 3: If you want to Identify words that are a way more frequent in spam than in non-spam, how are you going to modify the previous code?

->->

unique_words = set(list(spam_words.keys()) + list(non_spam_words.keys()))
spam_dominant_words = {}
for word in unique_words:
    spam_freq = spam_words[word]
    non_spam_freq = non_spam_words[word]

    p_w = spam_freq / spam_count
    q_w = non_spam_freq / non_spam_count
    if p_w > q_w:
        spam_dominant_words[word] = spam_freq
print("Words more frequent in spam than in non-spam:")
print(sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True))

14.	Question 4: How many words do you want to pick to make your spam filter? What are the words?

->-> 1821개
to
a
call


In [ ]:
def make_filter():
  p_num = 0
  q_num = 0
  p_w = 1.0
  q_w = 1.0
  p_total = 1.0
  q_total = 1.0
  r_w = 0.0
  spam_my_words={}
  for word, freq in sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True):
    p_num = spam_words[word] #the frequency of word in spam message  #sum(is_word_in_message(word, message) for message in spam_messages)
    q_num = non_spam_words[word] #the frequency of word in non-spam message #sum(is_word_in_message(word, message) for message in non_spam_messages)
    if q_num !=0:
      p_w = p_num / spam_count
      q_w = q_num / non_spam_count
      p_total *= p_w
      q_total *= q_w
      r_w = p_total / (p_total + q_total) #Generalized BAYES' Theorem
      if(r_w<1.0):
        spam_my_words[word] = freq
      else :
        break
      #print(word, p_total,q_total,p_num, q_num, r_w)
    else:
      continue
    #print(word, p_total,q_total, q_num, r_w)

  #print(len(spam_my_words))
  #print(spam_my_words)

  return spam_my_words

In [ ]:
# Function to check if a word is in a message
def is_word_in_message(word, message):
    message = message.lower()  # Convert to lowercase
    words = set(re.findall(r'\b\w+\b', message))  # Extract words and convert to a set
    return word in words


In [ ]:
# Specify the word to search for
search_word = '500'.lower()  # Replace with your specific word

# Filter spam messages
spam_messages = df[df['Label'] == 'spam']['Message']
print(len(spam_messages))

# Count how many spam messages contain the word
count = sum(is_word_in_message(search_word, message) for message in spam_messages)


# Print the result
print(f"The word '{search_word}' appears in {count} spam messages.")


747
The word '500' appears in 44 spam messages.


In [ ]:
#search_word = 'free'.lower() #Replace with your specific word
def create_word_frequency_dict(words): #Remove the duplicated words and calculate the number of words
    word_frequency_dict = {}

    for word in words:
        if word in word_frequency_dict:
            word_frequency_dict[word] += 1
        else:
            word_frequency_dict[word] = 1

    return word_frequency_dict


def make_filter():
  p_num = 0
  q_num = 0
  p_w = 0.0
  q_w = 0.0
  p_total = 1.0
  q_total = 1.0
  r_w = 0.0
  spam_my_words={}
  for word, freq in sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True):
    p_num = spam_words[word] #the frequency of word in spam message
    q_num = non_spam_words[word] #the frequency of word in non-spam message
    if q_num !=0:
      p_w = p_num / spam_count
      q_w = q_num / non_spam_count
      p_total *= p_w
      q_total *= q_w
      r_w = p_total / (p_total + q_total) #Generalized BAYES' Theorem
      if(r_w<1.0):
        spam_my_words[word] = freq
      else :
        break

    else:
      continue

  return spam_my_words

def is_spam(input_message):
  spam_my_words = make_filter() #make your spam-filter based on BAYES' Theorem

  p_num = 0
  q_num = 0
  p_w = 0.0
  q_w = 0.0
  p_total = 1.0
  q_total = 1.0
  r_w = 0.0

  for word in word_frequency_dict.keys():
    if word in spam_my_words:
      p_num = sum(is_word_in_message(word, message) for message in spam_messages)
      q_num = sum(is_word_in_message(word, message) for message in non_spam_messages)
      p_w = p_num / spam_count
      q_w = q_num / non_spam_count
      p_total *= p_w
      q_total *= q_w
    else:
      continue

    r_w = p_total / (p_total + q_total) #Generalized BAYES' Theorem
    print(word, p_total,q_total, r_w)
  return r_w > 0.9


input_message = input() #enter your specific message

# Tokenize the input message
tokenized_words = tokenize(input_message)

# Create a dictionary of word frequencies
word_frequency_dict = create_word_frequency_dict(tokenized_words)

if(is_spam(input_message)):
  print(input_message+" is spam message.")
else:
  print(input_message+" is not spam message.")

#'''
#winning to call me
#to 0.6251673360107095 0.25139896373056997 0.7132002863847595
#call 0.4390896921017403 0.04580310880829015 0.9597410014617073
#winning to call me is spam message.
#'''


I'm at work. Please call
call 0.4390896921017403 0.04580310880829015 0.9055397219296133
I'm at work. Please call is spam message.


In [ ]:
#search_word = 'free'.lower() #Replace with your specific word
def create_word_frequency_dict(words):
    word_frequency_dict = {}

    for word in words:
        if word in word_frequency_dict:
            word_frequency_dict[word] += 1
        else:
            word_frequency_dict[word] = 1

    return word_frequency_dict
def make_filter():
  p_num = 0
  q_num = 0
  p_w = 0.0
  q_w = 0.0
  p_total = 1.0
  q_total = 1.0
  r_w = 0.0
  spam_my_words={}
  for word, freq in sorted(spam_dominant_words.items(), key=lambda x: x[1], reverse=True):
    p_num = spam_words[word] #the frequency of word in spam message
    q_num = non_spam_words[word] #the frequency of word in non-spam message
    if q_num !=0:
      p_w = p_num / spam_count
      q_w = q_num / non_spam_count
      p_total *= p_w
      q_total *= q_w
      r_w = p_total / (p_total + q_total) #Generalized BAYES' Theorem
      if(r_w<1.0):
        spam_my_words[word] = freq
      else :
        break

    else:
      continue

  return spam_my_words

def is_spam(input_message):
  spam_my_words = make_filter() #make your spam-filter based on BAYES' Theorem

  p_num = 0
  q_num = 0
  p_w = 0.0
  q_w = 0.0
  p_total = 1.0
  q_total = 1.0
  r_w = 0.0

  for word in word_frequency_dict.keys():
    if word in spam_my_words:
      p_num = sum(is_word_in_message(word, message) for message in spam_messages)
      q_num = sum(is_word_in_message(word, message) for message in non_spam_messages)
      p_w = p_num / spam_count
      q_w = q_num / non_spam_count
      p_total *= p_w
      q_total *= q_w

    else:
      continue

  r_w = p_total / (p_total + q_total) #Generalized BAYES' Theorem

  return r_w > 0.9



k=1
for input_message in non_spam_messages:
  # Tokenize the input message
  tokenized_words = tokenize(input_message)
  # Create a dictionary of word frequencies
  word_frequency_dict = create_word_frequency_dict(tokenized_words)


  if(is_spam(input_message)==True):
    print(k, input_message+" is spam message.")
    k+=1


#if ((is_word_in_message(search_word, message) for message in spam_messages)):




#'''
#winning to call me
#to 0.6251673360107095 0.25139896373056997 0.7132002863847595
#call 0.4390896921017403 0.04580310880829015 0.9597410014617073
#winning to call me is spam message.
#'''


1 As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune is spam message.
2 I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times. is spam message.
3 I HAVE A DATE ON SUNDAY WITH WILL!! is spam message.
4 IÛ÷m going to try for 2 months ha ha only joking is spam message.
5 Did you catch the bus ? Are you frying an egg ? Did you make a tea? Are you eating your mom's left over dinner ? Do you feel my Love ? is spam message.
6 Wait that's still not all that clear, were you not sure about me being sarcastic or that that's why x doesn't want to live with us is spam message.
7 For fear of fainting with the of all that housework you just did? Quick have a cuppa is spam message.
8 Yup... Ok i go home look at the timings then i msg Ì_ again... Xuhui g

16.	Question 5: Using the multi-words filtering method covered in the class, implement a function that determines if a given input message is a spam or not. (Suppose the decision threshold is set to 90%.)

17.	Question 6: Change the number of words you used and test the performance.

18.	Question 7: Analyze(discuss) the advantages and disadvantages of the spam filter method implemented through this assignment.